In [1]:
from data import Data
from dimension_reduction import PCADimensionReduction
from autoencoder import Autoencoder

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

2022-11-27 23:16:04.196069: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_filepath = "data/SC_integration/counts_ctc_simulated_123_5k.tsv"
true_results_filepath = "data/SC_integration/ids_ctc_simulated_123_5k.tsv"
train_indices_filepath = "data/SC_integration/train_indices.npy"
test_indices_filepath = "data/SC_integration/test_indices.npy"

SEED = 42
FOLD_NUMBER = 3

# You can change these values to work better for models
CUT_BY_MAX_THRESHOLD = 4

In [3]:
data_object = Data(data_filepath, true_results_filepath)

train_data, test_data, train_true_results, test_true_results = data_object.load_train_test_split(train_indices_filepath, test_indices_filepath)

In [4]:
healthy_train_data, cancer_train_data, healthy_test_data, cancer_test_data = data_object.load_train_test_healthy_cancer_data()


In [5]:
def create_autoencoder(input_shape):
    model = Sequential()
    model.add(Dense(50, input_dim=input_shape, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(input_shape))
    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

## Autoencoder

### Regular data

In [6]:
autoencoder = create_autoencoder(healthy_train_data.shape[1])
autoencoder.fit(healthy_train_data, healthy_train_data, batch_size=16, epochs=100)

2022-11-27 23:16:07.369962: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 23:16:07.390011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 23:16:07.390363: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 23:16:07.391125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Epoch 1/100
190/190 [==============================] - 1s 1ms/step - loss: 0.1675
Epoch 2/100
  1/190 [..............................] - ETA: 0s - loss: 0.1378

2022-11-27 23:16:08.731201: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


190/190 [==============================] - 0s 1ms/step - loss: 0.1240
Epoch 3/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1200
Epoch 4/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1182
Epoch 5/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1170
Epoch 6/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1163
Epoch 7/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1158
Epoch 8/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1154
Epoch 9/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1148
Epoch 10/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1144
Epoch 11/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1142
Epoch 12/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1140
Epoch 13/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1138
Epoch 14/100
190/190 [==

In [7]:
autoencoder_object = Autoencoder(autoencoder, healthy_train_data, healthy_test_data, cancer_train_data, cancer_test_data)
autoencoder_object.display_average_rmse()

95/95 [==============================] - 0s 668us/step
Healthy train RMSE: 0.3266023540716223
Healthy test RMSE: 0.33310772158236784
Cancer train RMSE: 0.5041718876098047
Cancer test RMSE: 0.5097177980367162
Threshold: 0.4153871208407135


In [8]:
autoencoder_object.display_regular_metrics(train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(test_data, test_true_results, "Test")

97/97 [==============================] - 0s 717us/step
Train balanced accuracy: 0.9592535138438396
Train f1 score: 0.4647887323943662
Train precision: 0.308411214953271
Train recall: 0.9428571428571428
42/42 [==============================] - 0s 671us/step
Test balanced accuracy: 0.9800460475825019
Test f1 score: 0.36585365853658536
Test precision: 0.22388059701492538
Test recall: 1.0


### Scaled data

In [15]:
scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_train_data, scaled_cancer_test_data = data_object.get_scaled_healthy_train_test_cancer_data()
autoencoder = create_autoencoder(scaled_healthy_train_data.shape[1])
autoencoder.fit(scaled_healthy_train_data, scaled_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 1ms/step - loss: 0.9308
Epoch 2/100
190/190 [==============================] - 0s 1ms/step - loss: 0.9074
Epoch 3/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8999
Epoch 4/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8943
Epoch 5/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8892
Epoch 6/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8865
Epoch 7/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8843
Epoch 8/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8826
Epoch 9/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8806
Epoch 10/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8790
Epoch 11/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8771
Epoch 12/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8751
Epoch 13/100


In [16]:
autoencoder_object = Autoencoder(autoencoder, scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_train_data, scaled_cancer_test_data)
autoencoder_object.display_average_rmse()

95/95 [==============================] - 0s 702us/step
Healthy train RMSE: 0.897894064984329
Healthy test RMSE: 0.9413489714178648
Cancer train RMSE: 7.500427628127354
Cancer test RMSE: 7.014141306252867
Threshold: 4.199160846555841


In [11]:
scaled_train_data = data_object.scaled_data_by_healthy_train(train_data)
scaled_test_data = data_object.scaled_data_by_healthy_train(test_data)

autoencoder_object.display_regular_metrics(scaled_train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(scaled_test_data, test_true_results, "Test")

97/97 [==============================] - 0s 1ms/step
Train balanced accuracy: 0.9714285714285714
Train f1 score: 0.9705882352941176
Train precision: 1.0
Train recall: 0.9428571428571428
42/42 [==============================] - 0s 2ms/step
Test balanced accuracy: 0.9984650805832693
Test f1 score: 0.8823529411764706
Test precision: 0.7894736842105263
Test recall: 1.0


### Cut by max data

In [12]:
cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_train_data, cut_by_max_cancer_test_data = data_object.get_cut_by_max_healthy_train_test_cancer_data(CUT_BY_MAX_THRESHOLD)
autoencoder = create_autoencoder(cut_by_max_healthy_train_data.shape[1])
autoencoder.fit(cut_by_max_healthy_train_data, cut_by_max_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 1ms/step - loss: 1.6338
Epoch 2/100
190/190 [==============================] - 0s 1ms/step - loss: 0.5557
Epoch 3/100
190/190 [==============================] - 0s 1ms/step - loss: 0.5160
Epoch 4/100
190/190 [==============================] - 0s 1ms/step - loss: 0.4594
Epoch 5/100
190/190 [==============================] - 0s 1ms/step - loss: 0.4273
Epoch 6/100
190/190 [==============================] - 0s 1ms/step - loss: 0.4123
Epoch 7/100
190/190 [==============================] - 0s 1ms/step - loss: 0.4043
Epoch 8/100
190/190 [==============================] - 0s 930us/step - loss: 0.3990
Epoch 9/100
190/190 [==============================] - 0s 980us/step - loss: 0.3952
Epoch 10/100
190/190 [==============================] - 0s 1ms/step - loss: 0.3923
Epoch 11/100
190/190 [==============================] - 0s 924us/step - loss: 0.3905
Epoch 12/100
190/190 [==============================] - 0s 985us/step - loss: 0.3880
Epoch

In [13]:
autoencoder_object = Autoencoder(autoencoder, cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_train_data, cut_by_max_cancer_test_data)
autoencoder_object.display_average_rmse()

95/95 [==============================] - 0s 529us/step
Healthy train RMSE: 0.5871458358202771
Healthy test RMSE: 0.5972002817860236
Cancer train RMSE: 0.7562527516321903
Cancer test RMSE: 0.8357676152942657
Threshold: 0.6716992937262337


In [14]:
cut_by_max_train_data = data_object.cut_by_max_healthy_data(train_data, CUT_BY_MAX_THRESHOLD)
cut_by_max_test_data = data_object.cut_by_max_healthy_data(test_data, CUT_BY_MAX_THRESHOLD)

autoencoder_object.display_regular_metrics(cut_by_max_train_data, train_true_results, "Train")
autoencoder_object.display_regular_metrics(cut_by_max_test_data, test_true_results, "Test")

97/97 [==============================] - 0s 545us/step
Train balanced accuracy: 0.6963474827245805
Train f1 score: 0.061224489795918366
Train precision: 0.03225806451612903
Train recall: 0.6
42/42 [==============================] - 0s 545us/step
Test balanced accuracy: 0.8480941417242261
Test f1 score: 0.08284023668639053
Test precision: 0.043343653250773995
Test recall: 0.9333333333333333
